In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
sys.path.append("../src")

#Import config file. Update config.py according to your environment
from config import DATA_PATH, DRIVE_DATA_PATH, METADATA_PATH, TEXTDATA_PATH, PDFDATA_PATH

from dataset.utils import close_all_chrome_sessions
from dataset import load_dataset, push_data_to_drive


import os, glob, shutil
import subprocess



import pandas as pd
import numpy as np
from scipy import stats
import math

import requests, json, re

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from urllib3.exceptions import ReadTimeoutError
from selenium.common.exceptions import TimeoutException, WebDriverException
import psutil
import atexit

from pdfminer.high_level import extract_text, extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextLine, LTTextBox

from tqdm import tqdm
import time
from datetime import datetime
import multiprocessing
import operator
import tempfile



from pdfminer.pdfparser import PDFParser
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import logging
pdfminer_logger = logging.getLogger('pdfminer')
pdfminer_logger.setLevel(logging.ERROR)  

from pathlib import Path
import pickle
from science_parse_api.api import parse_pdf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
metadata, textdata = load_dataset(dataset_source='semanticsscholar', years=2005, data_types=['metadata', 'text'])
display(metadata)
display(textdata)

,paperId,openAccessPdf,fieldsOfStudy,venue,authorName,authorId,citationCount,influentialCitationCount,publicationDate,publicationYear,referenceIds,referenceTitles
0,0001d5fbff6f7c763a78dd5d141292416dcfae59,https://aacrjournals.org/clincancerres/article...,"Biology,Medicine",Clinical Cancer Research,"T. Ishibe,T. Nakayama,T. Okamoto,T. Aoyama,K. ...","14839966,10239302,2000448727,3201672,7147569,2...",88,6.0,2005-04-01,2005,,
1,000372290caae5482dcdd7954feb895ccbab921d,http://www.merl.com/publications/docs/TR2005-0...,"Computer Science,Mathematics",IEEE Transactions on Communications,"Juntan Zhang,M. Fossorier","3193350,48891339",319,26.0,2005-03-07,2005,17d0afe4f814e5abc6252e572e3b039082bf6f07;02464...,Low-Density Parity-Check Codes;Turbo decoder a...
2,000376ec6f6ade6261fc12df9eddd25a88c1e9ca,https://www.jstage.jst.go.jp/article/bpb/28/8/...,Medicine,Biological and Pharmaceutical Bulletin,"Bi-qi Zhang,Shen-Jiang Hu,Lihong Qiu,Q. Shan,J...","11809650,6617669,123635715,11305673,2152146096...",24,2.0,2005-08-01,2005,,
3,0003a5747ee3676be1b6d2c72c13a04e41811274,https://onlinelibrary.wiley.com/doi/pdfdirect/...,"Biology,Medicine",Developmental Dynamics,"Barden Chan,S. Sinha,Dan Cho,R. Ramchandran,V....","35690923,2114125286,2113252574,2134773,3658233",53,2.0,2005-01-01,2005,,
4,00043e28cffe744909bf69cde70e6dac0bce0a10,http://www.jbc.org/article/S0021925820589394/pdf,"Chemistry,Medicine",Journal of Biological Chemistry,"M. Westfall,A. M. Lee,D. Robinson","5025983,49249911,48706510",70,3.0,2005-12-16,2005,,
...,...,...,...,...,...,...,...,...,...,...,...,...
69988,fffb5a27f191307c8f2fed3a7d1162f93cbb8cb6,https://academic.oup.com/humrep/article-pdf/20...,"Medicine,Biology",Human Reproduction,"S. Doi,M. Al-Zaid,P. Towers,C. Scott,K. A. Al-...","40151220,1404195567,37556460,78745210,1403335938",46,0.0,2005-09-01,2005,3d2aaa390e58ff4d10cb2b6bfb4813e48565998c;8abd1...,Predictors for treatment failure after laparos...
69989,fffbc9b37f20515de2ff4fe60353d52dc37e65f5,http://www.cs.virginia.edu/~jl3aq/courses/CS85...,Computer Science,CCGrid 2005. IEEE International Symposium on C...,"A. Barak,A. Shiloh,Lior Amar","144214996,40399772,34848329",42,1.0,2005-05-09,2005,1713c96f258a9925b3ca0d814fdd0641c07bb3f1;3342b...,From sandbox to playground: dynamic virtual en...
69990,fffd2d6d54b4023ece28bfdebffdfb00502a09ec,https://europepmc.org/articles/pmc543423?pdf=r...,"Biology,Medicine",Molecular and Cellular Biology,"Ying Huang,R. Intine,Amy M Mozlin,Samuel A. Ha...","48355811,6534085,15003627,5662843,3935489",64,1.0,2005-01-01,2005,49bb10549894f0596e527139cf4a3bfb28fd6f2c;ced74...,Nuclear surveillance and degradation of hypomo...
69991,ffff785a92f7cacc7ecb6f3ac3f20ba91d5213b3,http://www.jbc.org/article/S0021925820789519/pdf,"Biology,Medicine",Journal of Biological Chemistry,"Faye Gordon,K. Luger,J. Hansen","48200695,3864710,34597814",143,3.0,2005-10-07,2005,83a02dee50f2f83987d5490c8c9501609fd8f2df;c2150...,Histone modifications: combinatorial complexit...


,paperId,title,abstract,referenceTitles,openAccessPdf
0,0001d5fbff6f7c763a78dd5d141292416dcfae59,Disruption of Fibroblast Growth Factor Signal ...,Purpose: Synovial sarcoma is a soft tissue sar...,,https://aacrjournals.org/clincancerres/article...
1,000372290caae5482dcdd7954feb895ccbab921d,Shuffled iterative decoding,Shuffled versions of iterative decoding of low...,Low-Density Parity-Check Codes;Turbo decoder a...,http://www.merl.com/publications/docs/TR2005-0...
2,000376ec6f6ade6261fc12df9eddd25a88c1e9ca,Diphasic effects of Astragalus membranaceus BU...,This study was designed to investigate the eff...,,https://www.jstage.jst.go.jp/article/bpb/28/8/...
3,0003a5747ee3676be1b6d2c72c13a04e41811274,Critical roles of CD146 in zebrafish vascular ...,"In this report, we use zebrafish as a model sy...",,https://onlinelibrary.wiley.com/doi/pdfdirect/...
4,00043e28cffe744909bf69cde70e6dac0bce0a10,Differential Contribution of Troponin I Phosph...,Cardiac troponin I is a phosphorylation target...,,http://www.jbc.org/article/S0021925820589394/pdf
...,...,...,...,...,...
69988,fffb5a27f191307c8f2fed3a7d1162f93cbb8cb6,Irregular cycles and steroid hormones in polyc...,BACKGROUND\nThis cross-sectional study was und...,Predictors for treatment failure after laparos...,https://academic.oup.com/humrep/article-pdf/20...
69989,fffbc9b37f20515de2ff4fe60353d52dc37e65f5,An organizational grid of federated MOSIX clus...,MOSIX is a cluster management system that uses...,From sandbox to playground: dynamic virtual en...,http://www.cs.virginia.edu/~jl3aq/courses/CS85...
69990,fffd2d6d54b4023ece28bfdebffdfb00502a09ec,Mutations in the RNA Polymerase III Subunit Rp...,ABSTRACT Termination by RNA polymerase III (Po...,Nuclear surveillance and degradation of hypomo...,https://europepmc.org/articles/pmc543423?pdf=r...
69991,ffff785a92f7cacc7ecb6f3ac3f20ba91d5213b3,The Core Histone N-terminal Tail Domains Funct...,Salt-dependent oligomerization of nucleosomal ...,Histone modifications: combinatorial complexit...,http://www.jbc.org/article/S0021925820789519/pdf


In [12]:
push_data_to_drive('metadata')

sending incremental file list

sent 1,710 bytes  received 33 bytes  3,486.00 bytes/sec
total size is 8,750,562,247  speedup is 5,020,402.90
Synced /home/jul/DST/asXiv/data/metadata -> /mnt/g/My Drive/DST/MLOps_projects/asXiv/data/metadata (rsync complete)


In [2]:
def load_dataset(path_to_data, dataset_source='semanticsscholar', years=None, data_types=None):
    """Fetch dataset metadata and text data for given years and data types."""
    years = (
        [str(year) for year in years] if isinstance(years, list) else
        [str(years)] if years else
        [year for year in os.listdir(os.path.join(path_to_data, 'metadata')) if year.isdigit()]
    )
    data_types = data_types or ['metadata', 'text']
    data_types = [data_types] if not isinstance(data_types, list) else data_types

    load_metadata = 'metadata' in data_types
    load_textdata = 'text' in data_types or 'pdf' in data_types
    load_pdfdata = 'pdf' in data_types

    metadata, textdata, pdfdata = [], [], []
    metadata_years, textdata_years, pdfdata_years = [], [], []

    for year in years:
        if load_metadata:
            metadata_path = os.path.join(path_to_data, 'metadata', year, f'{dataset_source}_metadata_{year}.parquet')
            if os.path.isfile(metadata_path):
                metadata.append(pd.read_parquet(metadata_path, engine="pyarrow"))
                metadata_years.append(year)
            else:
                print(f"No metadata for year {year}")
        
        if load_textdata:
            textdata_path = os.path.join(path_to_data, 'text', year, f'{dataset_source}_text_{year}.parquet')
            if os.path.isfile(textdata_path):
                textdata.append(pd.read_parquet(textdata_path, engine="pyarrow"))
                textdata_years.append(year)
            else:
                print(f"No text data for year {year}")

        if load_pdfdata:
            pdfdata_path = os.path.join(path_to_data, 'pdf', year, f'{dataset_source}_textpdf_{year}.parquet')
            if not os.path.isfile(pdfdata_path):
                pdfdata_path = os.path.join(path_to_data, 'text', year, f'{dataset_source}_text_{year}.parquet')
                print(f"No pdf data for year {year}. Will try to load text data instead")
            if os.path.isfile(pdfdata_path):
                pdfdata.append(pd.read_parquet(pdfdata_path, engine="pyarrow"))
                pdfdata_years.append(year)
            else:
                print(f"No pdf or text data for year {year}")
    
    metadata = pd.concat(metadata, axis=0, ignore_index=True).reset_index(drop=True) if metadata else []
    textdata = pd.concat(textdata, axis=0, ignore_index=True).reset_index(drop=True) if textdata else []
    pdfdata = pd.concat(pdfdata, axis=0, ignore_index=True).reset_index(drop=True) if pdfdata else []
    
    msg_parts = []
    if load_metadata:
        msg_parts.append(f'metadata loaded for years: {metadata_years}')
    if load_textdata:
        msg_parts.append(f'text data loaded for years: {textdata_years}')
    if load_pdfdata:
        msg_parts.append(f'pdf data loaded for years: {pdfdata_years}')
    
    if msg_parts:
        print("; ".join(msg_parts))
    if load_pdfdata:
        textdata = pdfdata.set_index("paperId").combine_first(textdata.set_index("paperId")).reset_index(drop=False)

    if load_metadata and load_textdata and len(metadata) != len(textdata):
        raise ValueError("Metadata and text data don't have the same length.")
    
    output = (data for data in [metadata, textdata] if len(data) > 0)
    return output


def push_data_to_drive(datatype: str, confirm: bool = True):
    valid_types = {
        "textdata": DATA_PATH / "textdata",
        "metadata": DATA_PATH / "metadata",
        "pdfdata": DATA_PATH / "pdfdata"
    }

    if datatype not in valid_types:
        raise ValueError(f"Invalid datatype '{datatype}'. Choose from: {list(valid_types.keys())}")

    source = valid_types[datatype]
    destination = DRIVE_DATA_PATH / datatype

    if confirm:
        confirm = input(f"Are you sure you want to sync {source} → {destination}? (Y/n): ").strip().lower()
        if confirm not in ['y', 'yes', '']:
            print("Sync aborted.")
            return
    
    subprocess.run([
        "rsync", "-rv", "--delete", "--size-only", #synch recursively and don't check timestamps
        str(source) + '/',  # ensures contents are copied, not the folder itself
        str(destination)
    ])
    print(f"Synced {source} -> {destination} (rsync complete, Google Drive sync in progress...)")

In [3]:
# pages = extract_pages("/home/jul/DST/recoRAG/data/temp/7/0b5101910aca09a90cdbc0281ad8f3f7638e0e61/1471-2105-7-364.pdf")
import pprint
from pathlib import Path
from science_parse_api.api import parse_pdf

pdf_path = Path("/home/jul/DST/recoRAG/data/temp/7/0b3b4465396deda32b6c74030afe241c59876063/PIIS0092867406004983.pdf").resolve()
host = 'http://127.0.0.1'
port = '8080'
output_dict = parse_pdf(host, pdf_path, port=port)

sections = {'pdf_abstract': output_dict['abstractText'],
        'pdf_introduction': ' '.join(s['text'] for s in output_dict['sections'] if 'heading' in s.keys() and 'introduction' in s['heading'].lower()),
        'pdf_results': ' '.join(s['text'] for s in output_dict['sections'] if 'heading' in s.keys() and 'results' in s['heading'].lower()),
        'pdf_discussion': ' '.join(s['text'] for s in output_dict['sections'] if 'heading' in s.keys() and 'discussion' in s['heading'].lower()),
        'pdf_methods': None,
        'pdf_text': ' '.join(s['text'] for s in output_dict['sections'] if 'text' in s.keys()),
        'author_list': ' / '.join(s['name'] for s in output_dict['authors'] if 'name' in s.keys()),
        'reference_titles': ' '.join(s['title'] for s in output_dict['references'] if output_dict['references'] and 'title' in s.keys())
        }

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(output_dict)

FileNotFoundError: [Errno 2] No such file or directory: '/home/jul/DST/recoRAG/data/temp/7/0b3b4465396deda32b6c74030afe241c59876063/PIIS0092867406004983.pdf'

In [ ]:
def get_pdf_num_pages(pdf_path):
    # Fast way to get total number of pages
    with open(pdf_path, 'rb') as f:
        return sum(1 for _ in PDFPage.get_pages(f))

def isvalid_page(pdf_path, page_idx):
    try:
        # extract_pages yields a single item here due to page_numbers=[page_idx]
        for _ in extract_pages(pdf_path, page_numbers=[page_idx]):
            return True  # Successfully processed page
    except Exception as e:
        return False  # Failed due to extract_pages exception
    return False

def get_valid_pages(pdf_path, timeout=None, num_processes=None):
    num_processes = 1 if num_processes is None else num_processes
    timeout = 0.1 if timeout is None else timeout
    valid_pages = []
    try:
        n_pages = get_pdf_num_pages(pdf_path)

        with multiprocessing.Pool(processes=num_processes) as pool:
            async_results = []
            # 1) Submit one task per page
            for i in range(n_pages):
                async_results.append(
                    pool.apply_async(isvalid_page, (pdf_path, i))
                )

            # 2) Collect results (with timeouts)
            for i, result in enumerate(async_results):
                try:
                    success = result.get(timeout=timeout)
                    if success:
                        valid_pages.append(i)
                except multiprocessing.TimeoutError:
                    # 3) If any page times out, forcibly kill the whole pool
                    pool.terminate()
                    break  # exit the for-loop
    except Exception as e:
        pdfminer_logger.warning(f"Error parsing PDF num pages for {pdf_path}: {e}")

    return valid_pages

In [4]:
downloads_dir = "/home/jul/DST/recoRAG/data/tmp/download-clps71gk/"
if os.path.isdir(downloads_dir):
        dir_list = os.listdir(downloads_dir)
        dir_list.sort(key=lambda name: os.path.getmtime(os.path.join(downloads_dir, name)))
        for dirname in dir_list:
            print(dirname)
            dir_path = os.path.join(downloads_dir, dirname)
            if glob.glob(os.path.join(dir_path, '*.pdf')):
                filepath = glob.glob(os.path.join(dir_path, '*.pdf'))[0]
                sections,_,_ = extract_pdf_sections(os.path.join(filepath), 'Bengtsson, Hossjer', dirname)

print(sections)

NameError: name 'sections' is not defined

In [4]:
def pdf_main_fontsize(pdf_path, valid_pages):
    try:
        sizes = [
        character.size
        for page_layout in extract_pages(pdf_path, page_numbers=valid_pages)
        for element in page_layout if isinstance(element, LTTextContainer)
        for text_line in element if isinstance(text_line, LTTextLine)
        for character in text_line if isinstance(character, LTChar)
    ]
        size_mode = stats.mode(sizes, keepdims=True)[0][0] if sizes else 0
    except Exception:
        size_mode = None
    return size_mode

def extract_pdf_sections(pdf_path, authorlist, paperId, valid_pages=None, possible_section_headings=None, size_threshold=None):
    author_last_names = [name.split()[-1] for name in authorlist.split(',') if name.split()]
    last_section_names = {'acknowledgment', 'acknowledgement', 'acknowlegment', 'reference'}
    month_markers = {'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'november', 'december',
                     'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'nov', 'dec'}
    year_markers = {str(y) for y in range(1900, 2030)}
    section_list = possible_section_headings or {
        'introduction', 'results', 'discussion', 'conclusions', 'methods', 'materials', 'experimental',
        'materials and methods', 'experimental procedure', 'related work', 'i.', 'ii.', 'iii.', 'iv.', 'v.', 'vi.'
    }

    size_threshold = size_threshold if size_threshold is not None else 0.9

    page_width, page_height = 595, 842
    nword_abstract_th, nword_sections_th = 30, 30

    if valid_pages == 0:
        sections = {'paperId': paperId,
            'pdf_abstract': None,
            'pdf_introduction': None,
            'pdf_results': None,
            'pdf_discussion': None,
            'pdf_methods': None,
            'pdf_text': None,
            'author_list': authorlist
            }
        return

    #Determine the most common font size (mode)
    size_mode = pdf_main_fontsize(pdf_path, valid_pages)

    text_blocks, tag_blocks, nwords_in_blocks = [], [], []
    section_to_find, section_heading, tag = 'AUTHORS', 'UNDEFINED', 'UNDEFINED'
    reached_end = False
    if size_mode is not None:
        for p, page_layout in enumerate(extract_pages(pdf_path, page_numbers=valid_pages)):
            if reached_end:
                break
            for element in page_layout:
                if not isinstance(element, LTTextBox):
                    continue
                x0, y0, x1, y1 = element.bbox
                if not (y0 > 0.05*page_height and y1 < 0.95*page_height and x0 > 0.05*page_width and x1 < 0.95*page_width):
                    continue

                filtered_text, sizes = [], []
                for text_line in element:
                    if isinstance(text_line, LTTextLine):
                        line_text = [character.get_text() for character in text_line]
                        sizes.extend([character.size for character in text_line if isinstance(character, LTChar)])
                        line_text = "".join(line_text).strip()
                        if not line_text.isdigit(): # Keeps lines that are NOT just numbers
                            filtered_text.append(line_text)
                        filtered_text.append("\n") # Preserve line breaks
                
                #joining characters in a single text while removing weird markers generated by pdfminer
                filtered_text = re.sub(r'\(cid:[^\)]+\)', '', "".join(filtered_text).strip())
                word_list = re.split(r'[\n\s]+', filtered_text.lower().strip())
                nwords = len(word_list)

                if any(end_section in ' '.join(word_list[:3]) for end_section in last_section_names):
                    reached_end = True
                    continue
                
                #removing everything before the author block as well as the correspondance fields
                if p <= 1:
                    nauthors_detected = sum(lastname.lower() in ' '.join(word_list) for lastname in author_last_names)
                    if nauthors_detected >= 0.5 * len(author_last_names) and y0 > 0.3 * page_height \
                    and section_to_find == 'AUTHORS':
                        text_blocks, tag_blocks, nwords_in_blocks = [], [], []
                        section_to_find = 'ABSTRACT'
                        filtered_text = []
                    if nauthors_detected > 0 and ('@' in filtered_text or "correspond" in filtered_text):
                        filtered_text = []

                #removing blocks likely headers with publication date
                if any([m in word_list for m in month_markers]) and any([y in word_list for y in year_markers]) and nwords < 10:
                    continue

                #removing figure captions
                if any([figname in word_list[0] for figname in ['fig', 'figure', 'table', 'image']]):
                    continue

                #removing previous block if likely a header but not followed by capitalized paragraph
                if filtered_text and not filtered_text[0].isupper() and nwords_in_blocks and nwords_in_blocks[-1] <= 3:
                    text_blocks.pop()
                    tag_blocks.pop()
                    nwords_in_blocks.pop()
                elif (filtered_text and filtered_text.strip() and filtered_text.strip()[0].isupper() and nwords_in_blocks and 
                    nwords_in_blocks[-1] <= 3 and text_blocks and any(h in re.sub(r'[\n\s]+', ' ', text_blocks[-1].lower()) for h in section_list)):
                    section_heading = ''.join([w.upper() for w in re.sub(r'[\d.]', '', text_blocks[-1])])
                    if nwords > nword_sections_th:
                        tag_blocks[-1] = section_heading

                if not reached_end and filtered_text and (max(sizes, default=0) >= size_threshold * size_mode or nwords > 50):
                    if section_to_find == 'ABSTRACT' and nwords > nword_abstract_th:
                        tag = 'ABSTRACT'
                        if word_list[-1][-1] == '.':
                            section_to_find = 'INTRODUCTION'
                    elif section_to_find == 'INTRODUCTION':
                        if nwords > nword_sections_th:
                            tag = 'INTRODUCTION'
                            section_heading, section_to_find = 'INTRODUCTION', 'NEXTHEADING'
                        else:
                            tag = 'UNDEFINED'
                    elif section_to_find == 'NEXTHEADING' and nwords > nword_sections_th:
                        tag = section_heading
                    
                    text_blocks.append(filtered_text)
                    tag_blocks.append(tag)
                    nwords_in_blocks.append(nwords)
    else:
        pdfminer_logger.warning(f"PDF {pdf_path} likely corrupted")

    sections = {'paperId': paperId,
            'pdf_abstract': '\n'.join(text for t, text in zip(tag_blocks, text_blocks) if any(s in t.lower() for s in {'abstract'})),
            'pdf_introduction': '\n'.join(text for t, text in zip(tag_blocks, text_blocks) if any(s in t.lower() for s in {'introduction', 'related work', 'i.', 'ii.'})),
            'pdf_results': '\n'.join(text for t, text in zip(tag_blocks, text_blocks) if any(s in t.lower() for s in {'results', 'experiment', 'i.', 'ii.'})),
            'pdf_discussion': '\n'.join(text for t, text in zip(tag_blocks, text_blocks) if any(s in t.lower() for s in {'discussion', 'conclusion', 'v.', 'vi.'})),
            'pdf_methods': '\n'.join(text for t, text in zip(tag_blocks, text_blocks) if any(s in t.lower() for s in {'methods', 'materials', 'experimental', 'materials and methods', 'experimental procedure'})),
            'pdf_text': '\n'.join(text for t, text in zip(tag_blocks, text_blocks) if not any(s in t.lower() for s in {'undefined'})),
            'author_list': authorlist
            }
    
    return sections, text_blocks, tag_blocks

In [5]:
# sudo apt-get update
# sudo apt-get install chromium-driver

def download_PDFs(txtdata):
    global driver
    #you'll need to have ChromeDriver installed
    downloads_dir = "/home/jul/DST/recoRAG/data/tmp"
    
    # service = Service(executable_path="/usr/bin/chromedriver")
    # run_headless = False #True #

    timeout_enddw = 10
    timeout_startdw = 5
    timeout_loadpage = 15 #timeout_startdw + timeout_enddw
    block_markers = {"captcha", "verify you are human", "not a robot", "forbidden", "error: 500", "this site can’t be reached"}
    error_markers = {'just a moment', 'privacy error', 'validate user', 'error 500'}

    # options = webdriver.ChromeOptions()
    # user_data_dir = tempfile.mkdtemp(prefix="chrome-profile-", dir="/tmp")
    # options.add_argument(f"--user-data-dir={user_data_dir}")
    # options.add_argument("--no-sandbox")
    # # options.add_argument("--disable-dev-shm-usage")

    # if run_headless:
    #     options.add_argument("--headless=new")
    #     options.add_argument("--disable-blink-features=AutomationControlled")
    # prefs = {
    #     "plugins.always_open_pdf_externally": True,
    #     "download.prompt_for_download": False,
    #     "download.default_directory": downloads_dir,
    #     "download.directory_upgrade": True
    # }
    # options.add_experimental_option("prefs", prefs)

    # waitforSession = 0
    # max_retries = 10
    # retry_delay = 3
    # while waitforSession < max_retries:
    #     try:
    #         driver = webdriver.Chrome(service=service, options=options)
    #         if run_headless:
    #             driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    #         break
    #     except Exception as e:
    #         print(f"Session creation failed. Retrying ({waitforSession+1}/{max_retries})...")
    #         time.sleep(retry_delay)  # Wait before retrying
    #         waitforSession += 1
    # else:
    #     raise RuntimeError(f"Failed to create a Selenium session after multiple attempts. Error: {e}")

    text_pdfs = []
    paperIds = []
    # fraction_print = 0
    # processed_list = []
    extensions = ('*.pdf', '*.crdownload')
    for k, paper in enumerate(txtdata):
        try:
            paper_download_dir = os.path.join(downloads_dir, paper['paperId'])
            os.makedirs(paper_download_dir, exist_ok=True)
            paperIds.append(paper['paperId'])

            # Set Chrome's download directory dynamically
            driver.execute_cdp_cmd(
                    "Page.setDownloadBehavior", 
                    {"behavior": "allow",
                    "downloadPath": paper_download_dir}
                    )
            clean_url = re.sub(r'^http://', 'https://', paper['openAccessPdf'])
            filename = None
            driver.set_page_load_timeout(timeout_loadpage)
            driver.get(clean_url)
            end_time_enddw = time.time() + timeout_enddw
            end_time_startdw = time.time() + timeout_startdw

            pdf_status = "download too long"
            
            if any(marker.lower() in driver.page_source.lower() for marker in block_markers) or \
                'crawlprevention' in driver.current_url or "europepmc.org/backend" in driver.current_url or \
                any(marker.lower() in driver.title.lower() for marker in error_markers):
                    pdf_status = driver.current_url
            else:
                while time.time() < end_time_startdw:
                    filename = [fname for ext in extensions for fname in glob.glob(os.path.join(paper_download_dir, ext))]
                    if filename:
                        pdf_status = 'downloading'
                        break
                    else:
                        time.sleep(0.2)

            # while time.time() < end_time_enddw:
            #     filename = [fname for ext in extensions for fname in glob.glob(os.path.join(paper_download_dir, ext))]
            #     if filename:
            #         # processed_list = [id for id in processed_list if not any(id in fname for fname in filename)]
            #         if len(filename) > 1:
            #             raise ImportError('more than one download to process: ', filename)
            #         filename = filename[0]
            #         break
            #     if any(marker.lower() in driver.page_source.lower() for marker in block_markers):
            #         filename = None
            #         break
            #     if time.time() > end_time_startdw and not glob.glob(os.path.join(paper_download_dir, '*.crdownload')):
            #         filename = None
            #         break
            # else:
            #     filename = None
            
        except Exception as e:
            pdf_status = "broken link"
            
        pdf_data = {'paperId': txtdata[k]['paperId'], 'author_list': paper['authorName'], 'pdf_text': None, 'pdf_abstract': None, 'pdf_introduction': None, 
                        'pdf_discussion': None, 'pdf_results': None, 'pdf_methods': None, 'pdf_status': pdf_status, 'openAccessPdf': paper['openAccessPdf'], 'valid_pages': None}
        text_pdfs.append(pdf_data)

    while download_inprogress(downloads_dir, paperIds):
        time.sleep(0.2)
        
    # driver.quit()
    # try:
    #     os.rmdir(user_data_dir)
    # except Exception:
    #     pass

    return text_pdfs

def download_inprogress(all_downloads_dir, dir_list):
    inprogress = False
    if os.path.isdir(all_downloads_dir):
        for dirname in dir_list:
            dir_path = os.path.join(all_downloads_dir, dirname)
            if os.path.isdir(dir_path):
                if glob.glob(os.path.join(dir_path, '*.crdownload')):
                    inprogress = True
                    break
    return inprogress
                
def get_valid_PDFs(text_pdfs, all_downloads_dir, timeout=None, num_processes=None):
    #Extracting pdfs from downloads that took too long to be processed immediately
    # downloads_dir = "/tmp/s2tmp"
    num_processes = 1 if num_processes is None else num_processes
    timeout = 0.1 if timeout is None else timeout
    if os.path.isdir(all_downloads_dir):
        dir_list = os.listdir(all_downloads_dir)
        dir_list.sort(key=lambda name: os.path.getmtime(os.path.join(all_downloads_dir, name)))
        for dirname in dir_list:
            dir_path = os.path.join(all_downloads_dir, dirname)
            if os.path.isdir(dir_path):
                if glob.glob(os.path.join(dir_path, '*.pdf')):
                    filepath = glob.glob(os.path.join(dir_path, '*.pdf'))[0]
                    paper_data = next((pdf_data for pdf_data in text_pdfs if pdf_data['paperId'] == dirname), None)
                    if paper_data:
                        valid_pages = get_valid_pages(filepath, timeout=timeout, num_processes=num_processes)
                        paper_data['valid_pages'] = valid_pages
    return text_pdfs

def get_unprocessed_PDFs():
    all_downloads_dir = "/home/jul/DST/recoRAG/data/tmp"
    unprocessed_PDFs = []
    if os.path.isdir(all_downloads_dir):
        dir_list = os.listdir(all_downloads_dir)
        for dirname in dir_list:
            try:
                dir_path = os.path.join(all_downloads_dir, dirname)
                if os.path.isdir(dir_path):
                    if glob.glob(os.path.join(dir_path, '*.pdf')):
                        unprocessed_PDFs.append(dirname)
            except Exception:
                pass
                    
    return unprocessed_PDFs

def gather_results(minibatches):
    downloads_dir = "/home/jul/DST/recoRAG/data/tmp"
    final_results = []
    for b in minibatches:
        mini_results = []
        for paper in b:
            dic_path = os.path.join(downloads_dir, paper['paperId'] + '.pkl')
            if os.path.isfile(dic_path):
                try:
                    with open(dic_path, 'rb') as f:
                        mini_results.append(pickle.load(f))
                    os.remove(dic_path)
                except Exception as e:
                    print(e)
                
        final_results.append(mini_results)
    return final_results
    
def extract_PDFs(textdata):
    #Extracting pdfs from downloads that took too long to be processed immediately
    # downloads_dir = "/tmp/s2tmp"
    timeout = 60
    downloads_dir = "/home/jul/DST/recoRAG/data/tmp"
    # dir_path_orig = [os.path.join(downloads_dir, pdf_data['paperId']) for pdf_data in textdata]
    dir_list = [{'id': pdf_data['paperId'], 'time': None} for pdf_data in textdata]
    while dir_list:
        # if os.path.isdir(downloads_dir):
        #     dir_list = os.listdir(downloads_dir) if dir_list is None else dir_list
        # dir_list.sort(key=lambda name: os.path.getmtime(os.path.join(downloads_dir, name)))
        for dirname in dir_list:
            dir_path = os.path.join(downloads_dir, dirname['id'])
            if os.path.isdir(dir_path):
                if not dirname['time']:
                    dirname['time'] = os.path.getmtime(dir_path)
                if glob.glob(os.path.join(dir_path, '*.pdf')):
                    filepath = glob.glob(os.path.join(dir_path, '*.pdf'))[0]
                    paper_data = next((pdf_data for pdf_data in textdata if pdf_data['paperId'] == dirname['id']), None)
                    if paper_data:
                        if 'valid_pages' not in paper_data.keys():
                            paper_data['valid_pages'] = None
                        pdf_data,_,_ = extract_pdf_sections(filepath, paper_data['authorName'], paper_data['paperId'], paper_data['valid_pages'])
                        for key in pdf_data.keys():
                            paper_data[key] = pdf_data[key]
                        paper_data['pdf_status'] = "extracted"

                        dic_path = os.path.join(downloads_dir, dirname['id'] + '.pkl')
                        with open(dic_path, 'wb') as f:
                            pickle.dump(paper_data, f, protocol=pickle.HIGHEST_PROTOCOL)

                    dir_list.remove(dirname)
                    shutil.rmtree(dir_path)
                elif time.time() - dirname['time'] > timeout:
                    crfilepath = glob.glob(os.path.join(dir_path, '*.crdownload'))
                    if not crfilepath or time.time() - os.path.getmtime(crfilepath[0]) > timeout:
                        dir_list.remove(dirname)
                        shutil.rmtree(dir_path)

    # for dir_path in dir_path_orig:
    #     if os.path.isdir(dir_path):
    #         shutil.rmtree(dir_path)

    # if not glob.glob(os.path.join(downloads_dir, '*')):
    #     shutil.rmtree(downloads_dir)

    return textdata

In [6]:
# def close_all_chrome_sessions():
#     nkill = 0
#     for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
#         if proc.info['name'] and ('chrome' in proc.info['name'].lower() or 
#                                   'chromium' in proc.info['name'].lower()):
#             try:
#                 proc.kill()
#                 nkill += 1
#             except Exception as e:
#                 print(f"Could not close process {proc.info['pid']}: {e}")
#     print(f"Closed {nkill} Chrome processes")

def download_worker_init():
    global driver, user_data_dir

    worker_id = os.getpid()
    user_data_dir = tempfile.mkdtemp(prefix=f"chrome-profile-{worker_id}", dir="/tmp")

    downloads_dir = "/home/jul/DST/recoRAG/data/tmp"
    
    service = Service(executable_path="/usr/bin/chromedriver")
    run_headless = False #True #

    options = webdriver.ChromeOptions()
    options.add_argument(f"--user-data-dir={user_data_dir}")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=400,300")
    options.add_argument("--window-position=10,10")
    # options.add_argument("--disable-dev-shm-usage")

    if run_headless:
        options.add_argument("--headless=new")
        # options.add_argument("--disable-blink-features=AutomationControlled")
    prefs = {
        "plugins.always_open_pdf_externally": True,
        "download.prompt_for_download": False,
        "download.default_directory": downloads_dir,
        "download.directory_upgrade": True
    }
    options.add_experimental_option("prefs", prefs)

    waitforSession = 0
    max_retries = 10
    retry_delay = 3
    while waitforSession < max_retries:
        try:
            driver = webdriver.Chrome(service=service, options=options)
            # if run_headless:
            #     driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
            break
        except Exception as e:
            print(f"Session creation failed. Retrying ({waitforSession+1}/{max_retries})...")
            time.sleep(retry_delay)  # Wait before retrying
            waitforSession += 1
    else:
        raise RuntimeError(f"Failed to create a Selenium session after multiple attempts")

    time.sleep(0.5)
    atexit.register(download_worker_close)

def download_worker_close():
    global driver, user_data_dir
    worker_id = os.getpid()
    
    driver.quit()
    print(f"Worker {worker_id} WebDriver closed.")
    
    shutil.rmtree(user_data_dir, ignore_errors=True)
    print(f"Worker {worker_id} data dir {user_data_dir} deleted.")

def download_and_extract_PDFs(path_to_data, dataset_source=None, years=None, n_jobs=None, filters=None,
                              timeout_per_article=None):
    dataset_source = 'semanticsscholar' if dataset_source is None else dataset_source
    years = [datetime.now().year] if years is None else years
    years = [years] if not isinstance(years, list) else years

    output_dir = os.path.join(path_to_data, 'pdfdata')
    os.makedirs(output_dir, exist_ok=True)

    #Cleaning-up tmp dir
    for dirname in os.listdir("/tmp"):
        if "chrome-profile-" in dirname:
            shutil.rmtree(os.path.join("/tmp", dirname))
    
    downloads_dir = "/home/jul/DST/recoRAG/data/tmp"
    os.makedirs(downloads_dir, exist_ok=True)
    

    # filters = [['abstract','==','None']]
    filters = [filters] if filters is not None and not isinstance(filters[0], list) else filters

    n_jobs = multiprocessing.cpu_count() if n_jobs is None else n_jobs
    n_jobs_dwload, n_jobs_extract = (n_jobs // 2, n_jobs // 2) if not isinstance(n_jobs, list) else tuple(n_jobs)
    n_jobs_total = n_jobs_dwload + n_jobs_extract

    timeout_per_article = 0.4 if timeout_per_article is None else timeout_per_article
    timeout_per_worker = timeout_per_article * sum((n_jobs_dwload, n_jobs_extract))

    ops = {
            "==": operator.eq,
            "!=": operator.ne,
            ">": operator.gt,
            "<": operator.lt,
            ">=": operator.ge,
            "<=": operator.le,
            "isna": pd.isna,
            "notna": pd.notna
        }

    for year in years:
        print(f"Downloading pdfs for year {year}")
        metadata, textdata = load_dataset(path_to_data, dataset_source, years=year, data_types=['metadata', 'pdf'])
        filtered_indices = textdata.index
        if filters:
            for col, op, val in filters:
                if col in textdata.columns:
                    if op in ["isna", "notna"]:
                        idx_list = textdata[ops[op](textdata[col])].index
                    else:
                        idx_list = textdata[ops[op](textdata[col], val)].index

                    filtered_indices = filtered_indices.intersection(idx_list)

        data4pdf = pd.merge(textdata[['paperId','openAccessPdf']].loc[filtered_indices], metadata[['paperId','authorName']].loc[filtered_indices], on='paperId', how='inner')
        if len(data4pdf) < len(filtered_indices):
            raise ValueError("List of metadata and text data are notmatching the same articles")
        
        # data4pdf = data4pdf.iloc[:400]
        minibatch_size = 20 #100
        batch_size = 3000 #len(data4pdf) // 10 #2 * n_jobs * minibatch_size
        Npapers = 3000 #len(data4pdf)
        with tqdm(range(Npapers // batch_size + 1), desc=f'Year {year} / {len(years)} years ({Npapers} papers on that year)', unit='batch') as pbar:
            try:
                for i in pbar:
                    shutil.rmtree(downloads_dir)
                    os.makedirs(downloads_dir, exist_ok=True)
                    close_all_chrome_sessions()
                    
                    TimoutRaised = False
                    batch = data4pdf.iloc[i*batch_size:min(Npapers, (i+1)*batch_size)]
                    Nbatch = len(batch)
                    minibatches = [batch.iloc[k*minibatch_size:min(Nbatch, (k+1)*minibatch_size)].to_dict(orient='records') for k in range(Nbatch // minibatch_size + 1)]
                    minibatches = [b for b in minibatches if len(b) > 0 and b[0] is not None]
                    final_results, dwload_results = [], []
                    Nminibatches = len(minibatches)
                    if minibatches:
                        # with multiprocessing.Pool(processes=n_jobs) as pool:
                        #     results = list(pool.map(download_PDFs, minibatches))
                        with multiprocessing.Pool(processes=n_jobs_dwload, initializer=download_worker_init) as download_pool, \
                             multiprocessing.Pool(processes= n_jobs_extract) as extract_pool:

                            async_dwload = [download_pool.apply_async(download_PDFs, (b,)) for b in minibatches]  
                            async_extract = [extract_pool.apply_async(extract_PDFs, (b,)) for b in minibatches]
                            
                            pending_extract = set(async_extract)
                            pending_dwload = set(async_dwload)
                            end_time = time.time() + timeout_per_article*batch_size
                            old_pending_articles = batch_size
                            while pending_extract or pending_dwload:
                                #Adjust timeout wall to one many tasks of extraction are still pending still download is done
                                if time.time() > end_time:
                                    print(f"{len(pending_extract)} extractions and {len(pending_dwload)} downloads out of {Nminibatches} timed out. Terminating the pool.")
                                    TimoutRaised = True
                                    extract_pool.terminate()
                                    extract_pool.join()
                                    download_pool.terminate()
                                    download_pool.join()
                                    break

                                for task in list(pending_extract):
                                    if task.ready():
                                        res = task.get()
                                        final_results.append(res)
                                        pending_extract.remove(task)

                                for task in list(pending_dwload):
                                    if task.ready():
                                        res_dw = task.get()
                                        dwload_results.append(res_dw)
                                        pending_dwload.remove(task)

                                if pending_extract or pending_dwload:
                                    if pending_dwload:
                                        pbar.set_postfix({'download tasks': f"{len(pending_dwload)} left",
                                                        'extraction tasks': f"{len(pending_extract)} left"})
                                    time.sleep(1)

                                if not pending_dwload:
                                    n_pending_articles = len(get_unprocessed_PDFs())
                                    if n_pending_articles != old_pending_articles:
                                        extratime = timeout_per_worker / 2 * n_pending_articles / len(pending_extract)
                                        pbar.set_postfix({'extraction tasks': f"{len(pending_extract)} still running, \
                                                          {extratime} seconds left"})
                                        end_time = time.time() + extratime
                                        old_pending_articles = n_pending_articles
                                    else:
                                        end_time = time.time()
                        
                        pbar.set_postfix({'status': "gathering results..."})
                        final_results = gather_results(minibatches)
                        
                        if TimoutRaised:
                            #Checking valid pages to avoid pdfminer getting stuck
                            #This will change results in place
                            n_pending_articles = len(get_unprocessed_PDFs())
                            pbar.set_postfix({'status': f"checking valid pages for {n_pending_articles} articles that timed out..."})
                            get_valid_PDFs([pdf for r in minibatches for pdf in r], downloads_dir, timeout=1)
                            lo_ids = get_unprocessed_PDFs()
                            lo_papers = [paper for b in minibatches for paper in b if paper['paperId'] in lo_ids]
                            N_lo = len(lo_papers)
                            minib_size = math.ceil(len(lo_papers) / (n_jobs_total + 1))
                            lo_minibatches = [lo_papers[k*minib_size:(k+1)*minib_size] for k in range(n_jobs_total + 1)]
                            lo_minibatches = [b for b in lo_minibatches if len(b) > 0 and b[0] is not None]
                            Nlo_minibatches = len(lo_minibatches)
                            pbar.set_postfix({'status': f"will process now {(N_lo)} articles..."})
                            print()
                            with multiprocessing.Pool(processes=n_jobs_total) as pool:
                                async_extract = [pool.apply_async(extract_PDFs, (b,)) for b in lo_minibatches]
                                pending = set(async_extract)
                                end_time = time.time() + timeout_per_article*N_lo
                                while pending:
                                    if time.time() > end_time:
                                        print(f"{len(pending)} tasks out of {Nlo_minibatches} timed out after checking valid pdf pages.")
                                        TimoutRaised = True
                                        pool.terminate()
                                        pool.join()
                                        break

                                    for task in list(pending):
                                        if task.ready():
                                            res = task.get()
                                            final_results.append(res)
                                            pending.remove(task)
                                    if pending:
                                        time.sleep(0.5)

                        if final_results:                
                            pdfdata = pd.DataFrame([pdf for r in final_results for pdf in r])
                            pbar.set_postfix({'status': f"{len(pdfdata)} articles extracted out of {batch_size}"})
                            if dwload_results:
                                dwdata = pd.DataFrame([{k: d.get(k, None) for k in ['paperId', 'pdf_status']}
                                                    for r in dwload_results if r is not None 
                                                    for d in r if d is not None])
                                pdfdata = dwdata.set_index("paperId").combine_first(pdfdata.set_index("paperId")).reset_index(drop=False)
                            textdata = pdfdata.set_index("paperId").combine_first(textdata.set_index("paperId")).reset_index(drop=False)
                            mask = textdata['abstract'] == 'None'
                            textdata.loc[mask, 'abstract'] = textdata.loc[mask, 'pdf_abstract']
                            textdata['authorName'] = metadata['authorName']

                # Cleaning up download directory
                shutil.rmtree(downloads_dir)
                os.makedirs(downloads_dir, exist_ok=True)
                close_all_chrome_sessions()
                            
            except Exception as e:
                pbar.close()
                raise


        output_dir_year = os.path.join(output_dir, str(year))
        os.makedirs(output_dir_year, exist_ok=True)
        filepath = os.path.join(output_dir_year, f'{dataset_source}_textpdf_{year}.parquet')

        # textdata.to_parquet(filepath, engine="pyarrow", compression="snappy", index=True)

    return textdata, pdfdata

In [3]:
push_data_to_drive('metadata')

sending incremental file list
2005/
2005/semanticsscholar_metadata_2005.parquet

sent 162,843,845 bytes  received 59 bytes  36,187,534.22 bytes/sec
total size is 8,750,562,247  speedup is 53.74
Synced /home/jul/DST/asXiv/data/metadata -> /mnt/g/My Drive/DST/MLOps_projects/asXiv/data/metadata (rsync complete, Google Drive sync in progress...)


In [7]:
textdata, pdfdata = download_and_extract_PDFs(DATA_PATH, years=2006, n_jobs=[20, 30])#, filters=[['abstract','==','None']])

No pdf data for year 2006. Will try to load text data instead
metadata loaded for years: ['2006']; text data loaded for years: ['2006']; pdf data loaded for years: ['2006']


Year 2006 / 1 years (3000 papers on that year):   0%|          | 0/2 [00:00<?, ?batch/s]

Closed 0 Chrome processes


Year 2006 / 1 years (3000 papers on that year):   0%|          | 0/2 [11:42<?, ?batch/s, extraction tasks=10 still running,                                                           5.0 seconds left]

10 extractions and 0 downloads out of 150 timed out. Terminating the pool.


Year 2006 / 1 years (3000 papers on that year):   0%|          | 0/2 [11:49<?, ?batch/s, status=will process now 5 articles...]                                                                        

Year 2006 / 1 years (3000 papers on that year):   0%|          | 0/2 [11:53<?, ?batch/s, status=1713 articles extracted out of 3000]

1 tasks out of 5 timed out after checking valid pdf pages.


Year 2006 / 1 years (3000 papers on that year): 100%|██████████| 2/2 [11:54<00:00, 357.31s/batch, status=1713 articles extracted out of 3000]

Closed 369 Chrome processes
Closed 168 Chrome processes


In [11]:
textdata.iloc[:3000].isna().sum()

paperId                0
abstract             129
authorName             0
author_list         1225
openAccessPdf          0
pdf_abstract        1225
pdf_discussion      1225
pdf_introduction    1225
pdf_methods         1225
pdf_results         1225
pdf_status             0
pdf_text            1225
referenceTitles        0
title                  0
valid_pages         2995
dtype: int64

In [ ]:
metadata0, textdata0 = load_dataset(DATA_PATH, years=[2006], data_types=['metadata', 'pdf'])

No pdf data for year 2006. Will try to load text data instead
metadata loaded for years: ['2006']; text data loaded for years: ['2006']; pdf data loaded for years: ['2006']


In [24]:
textdata0

,paperId,title,abstract,referenceTitles,openAccessPdf
0,00009f98f5ff962beb322e6c0e78e4b77dd2d968,"High exposure to Tunga penetrans (Linnaeus, 17...",Tungiasis is a parasitic skin disease widespre...,,https://www.scielo.br/j/mioc/a/csnpsRLq5FqY5Mk...
1,0000e1833852eb3271356c493d59e83da0c4c7db,Immunoinformatics Comes of Age,With the burgeoning immunological data in the ...,Prediction of the secondary structure of prote...,https://journals.plos.org/ploscompbiol/article...
2,00016363b9a128e01a17e9b831249967a0fee15f,The Potassium Channel KAT1 Is Activated by Pla...,14-3-3 proteins modulate the plant inward rect...,,http://www.jbc.org/article/S0021925820681212/pdf
3,00034b38a5273b760824f3223fa0d4fdb349d6df,Competitive recruitment of the periplasmic tra...,The natively disordered N-terminal 83-aa trans...,,https://www.pnas.org/doi/pdf/10.1073/pnas.0603...
4,00036a3493967932ac256f38c0c2419415f04feb,Polysaccharides from the root of Angelica sine...,Cyclophosphamide (CY) is a cytostatic agent th...,,https://www.medsci.org/v03p0001.pdf
...,...,...,...,...,...
76645,fffabc38e25edd5a95b48e2952322542317ea392,Implicit vs. Explicit Data-Flow Requirements i...,None,Specifying Data-Flow Requirements for the Auto...,https://link.springer.com/content/pdf/10.1007%...
76646,fffae8f127ca823fecd1228e5131aa06ec98696a,Presence of Pathogenicity Island Genes in Ente...,ABSTRACT Enterococcus faecalis isolates of por...,Multilocus Sequence Typing Scheme for Enteroco...,https://europepmc.org/articles/pmc1698369?pdf=...
76647,fffe4813ee529996e47ec6c4f1732f0030d334c9,Variability of Motor Neuron Spike Timing Maint...,The accessory radula closer (ARC) muscle of Ap...,Tight or loose coupling between components of ...,https://www.jneurosci.org/content/jneuro/26/26...
76648,ffff1780c2f0bb4d8ea115fdd820a518a1b33dd6,Relationship between Thermodynamic Driving For...,Chemical reaction systems operating in nonequi...,Generalized Haldane equation and fluctuation t...,https://journals.plos.org/plosone/article/file...


In [34]:
# Sample DataFrames
df1 = pd.DataFrame({
    "id": [1, 2, 3],
    "name": ["Alice", "Bob", "Charlie"],
    "age": [25, 30, 35],
})

df2 = pd.DataFrame({
    "id": [3, 4],  # id=2 and id=3 exist, id=4 is new
    "name": ["Charlie", "David"],  # id=3 has new data
    "age": [36, 40],  # Updates for id=2 and id=3
    "taille": [170, 180]
})

# 🔥 Merge and update existing rows
merged_df = df2.set_index("id").combine_first(df1.set_index("id")).reset_index(drop=False)

print(merged_df)

   id  age     name  taille
0   1   25    Alice     NaN
1   2   30      Bob     NaN
2   3   36  Charlie   170.0
3   4   40    David   180.0
